In [30]:
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

from fastapi import FastAPI, HTTPException, File, UploadFile
import uvicorn
from pydantic import BaseModel



# Later, in another script or session, you can load the scaler
loaded_scaler = joblib.load('standard_scaler.pkl')
# Load the saved autoencoder model
loaded_autoencoder = load_model("autoencoder_model.h5")
# Load Featurewise Thresold
feature_wise_threshold = joblib.load("feature_wise_thresold.pkl")
# Thresold for reconstruction loss
threshold = 0.0005

In [48]:
test_df = pd.read_csv(r'Dataset/Fault_1.csv')

x  = test_df.iloc[10,3:].values.tolist()
x = loaded_scaler.transform([x])
x_recon = loaded_autoencoder.predict(x)

reconstruction_loss = np.mean((x-x_recon)**2,axis=1)/ threshold
feature_wise_recon = np.abs(x-x_recon)/feature_wise_threshold

1/1 [==============================] - 0s 53ms/step


c:\Users\Administrator\miniconda3\envs\TEP_deployment\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [52]:
# Create a FastAPI instance
app = FastAPI()

# Define the input data model using Pydantic
class InputData(BaseModel):
    features: list[float]

# Define a predict endpoint
@app.post("/predict")
def predict(data: InputData):
    # Access the input data
    input_values = data.features

    # Transform the input data using the loaded scaler
    x = loaded_scaler.transform([input_values])

    # Predict using the loaded autoencoder
    x_recon = loaded_autoencoder.predict(x)

    # Calculate reconstruction loss
    reconstruction_loss = np.mean((x - x_recon) ** 2, axis=1) / threshold

    # Calculate feature-wise reconstruction
    feature_wise_recon = np.abs(x - x_recon) / feature_wise_threshold

    # Return the results
    return {"reconstruction_loss": reconstruction_loss.tolist(), "feature_wise_recon": feature_wise_recon[0].tolist()}

In [53]:
from fastapi.testclient import TestClient
# Test your endpoint using the TestClient

client = TestClient(app)


x_test  = test_df.iloc[10,3:].values.tolist()
input_data = {"features": x_test}  # Provide the actual list of features

# Send a POST request to the /predict endpoint
response = client.post("/predict", json=input_data)



# Print the response JSON
print(response.json())

1/1 [==============================] - 0s 25ms/step
{'reconstruction_loss': [0.1591294144058924], 'feature_wise_recon': [0.014048488366402665, 0.07167946043628737, 0.011968706024474815, 0.0015770186244844732, 0.029283588173224417, 0.006355635165405297, 0.11225035999592765, 0.033926212173757554, 0.01419359483124265, 0.026797744614696663, 0.04238606064262542, 0.09961963263940715, 0.031380712923322764, 0.034281741101467596, 0.04925037944707725, 0.15324045735262895, 0.031007532593777747, 0.152770774787654, 0.06025367743126983, 0.030583365375436055, 0.056213125667079464, 0.03034458768844238, 0.019222428499781712, 0.027514890704489765, 0.06856704697278396, 0.01809739019157608, 0.01873914649719986, 0.014364214013638346, 0.04835392928811603, 0.045712765830323465, 0.012217279599609514, 0.05724062558367255, 0.05670062964287344, 0.04569547025749758, 0.044976663065973116, 0.01367978819346021, 0.002169723375035531, 0.0597417429964809, 0.004326707888322666, 0.01528089147194902, 0.05934321568835971, 

c:\Users\Administrator\miniconda3\envs\TEP_deployment\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
